This challenge implements an instantiation based on LWE where the error in discrete Gaussian distribution(see [[1]](#1), [[2]](#2)), a Search-LWE problem can be written as:

$$\mathbf{A}\cdot \mathbf{s} + \mathbf{e}\equiv \mathbf{b}\pmod q$$

where $\mathbf{A} \in \mathbb{Z}^{n\times m}_q$, $\mathbf{s} \in \mathbb{Z}^{n}_q$ and $\mathbf{e} \in \mathbb{Z}^{m}_q$, our goal is to find $\mathbf{s}$ given $(\mathbf{A}, \mathbf{b})$, but in this challenge both $\mathbf{A}$ and $\mathbf{b}$ are masked by some random vectors or matrix, we must solve some lattice-based problems to recover them at first.

For $\mathbf{b}$, the main problem is:

$$\mathbf{T}\cdot \mathbf{b} = \mathbf{R}, \mathbf{T}\in \mathbb{Z}^{1\times 64}_{2^{1024}}, \mathbf{b}\in \mathbb{Z}^{64\times 1}_{1000}$$

We need to recover $\mathbf{b}$ given $\mathbf{T}$ and $\mathbf{R}$, we can regard this problem as a knapsack-like problem, since the value of the elements in $\mathbf{b}$ is much smaller than in $\mathbf{T}$, it can be done easily using LLL-reduction:

In [1]:
T = Matrix([randint(1, 2^1024) for _ in range(64)])
b = vector([randint(1, 1000) for _ in range(64)])
R = T * b

res = T.transpose().stack(R).transpose().right_kernel_matrix()
res = res.LLL()
b == vector(map(abs, res[0][:-1]))

True

For $\mathbf{A}$, the main problem is:

$$\mathbf{A}\cdot \mathbf{T} = \mathbf{R}, \mathbf{A}\in \mathbb{Z}^{320\times 5}_{1000}, \mathbf{T}\in \mathbb{Z}^{5\times 7}_{2^{1024}}$$

We need to recover $\mathbf{A}$ only given $\mathbf{R}$, this situation becomes a bit more complicated since this time we only know the product of two unknown matrix, though it may seem impossible, we can recover it since the value of the elements in $\mathbf{A}$ is much smaller than in $\mathbf{T}$, noticed that each row vector in $\mathbf{T}^\top$ is a linear combination of all row vectors in $\mathbf{A}^\top$, we can perform LLL-reduction on $\mathbf{T}$, then check whether there is a vector in the linear combination of the obtained vectors whose values of all elements fall in the interval of $[10, 1000]$, if it exists, we mark it as a candidate vector, after finding five candidates we only need to enumerate in its full permutation:

In [2]:
import itertools

def check(l):
    return sum([i>=10 and i<=1000 for i in l]) == 320 or sum([-i>=10 and -i<=1000 for i in l]) == 320

def get_A(res, idx_list, op_list):
    A_cols = []
    for a, b, c, d, e in idx_list:
        for op in op_list:
            v = res[a]*op[0] + res[b]*op[1] + res[c]*op[2] + res[d]*op[3] + res[e]*op[4]
            if check(v) and (v not in A_cols) and (-v not in A_cols):
                A_cols.append(v)
            if len(A_cols) == 5:
                return A_cols

A = random_matrix(ZZ, 320, 5, x = 10, y = 1000)
R = Matrix(A * vector([randint(1, 2^1024) for _ in range(5)]) for _ in range(7))

res = R.LLL()

idx_list = list(cartesian_product([[2, 3, 4, 5, 6] for _ in range(5)]))
op_list = list(cartesian_product([[-1, 0, 1] for _ in range(5)]))

ans = get_A(res, idx_list, op_list)
ans = [i if i>0 else -i for i in ans]

possible_A  = list(map(Matrix, list(itertools.permutations(ans))))
possible_A  = [i.transpose() for i in possible_A]
A in possible_A

True

Now the only thing left is to solve a Search-LWE problem, LWE is as hard as classical lattice problems, such as the SIVP, in the worst case. But in practice, there exist many attacks for LWE in different model(see [[3]](#3), Section 3.3;[[4]](#4), Section 4). In this challenge, the security parameter $n$ are suitably small, we can convert the LWE problem to a CVP problem and applied LLL or BKZ lattice basis reduction:

In [3]:
from sage.crypto.lwe import LWE
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler as DGDIS

L = LWE(n = 25, q = 1000, D = DGDIS(3))
S = [L() for _ in range(64)]
A = matrix([x for x, _ in S])
b = vector(ZZ, [y for _, y in S])

basis = A.transpose().change_ring(ZZ).stack(1000 * identity_matrix(64)).hermite_form()[:64]
res = block_matrix([[matrix(ZZ,1,1,[3]), matrix(b)], [matrix(ZZ, 64, 1, [0] * 64), basis]])

res = res.LLL()
e = res[0][1:]
s = A \ (b - e)

s == L._LWE__s

True

Here is my final solver:

In [4]:
import itertools
from Crypto.Cipher import AES
from Crypto.Util.number import *
from hashlib import sha256

def check(l):
    return sum([i>=10 and i<=1000 for i in l]) == 320 or sum([-i>=10 and -i<=1000 for i in l]) == 320
                                    
def get_A(res, idx_list, op_list):
    A_cols = []
    for a, b, c, d, e in idx_list:
        for op in op_list:
            v = res[a]*op[0] + res[b]*op[1] + res[c]*op[2] + res[d]*op[3] + res[e]*op[4]
            if check(v) and (v not in A_cols) and (-v not in A_cols):
                A_cols.append(v)
            if len(A_cols) == 5:
                return A_cols

def get_s(A, b):
    basis = A.transpose().change_ring(ZZ).stack(1000 * identity_matrix(64)).hermite_form()[:64]
    res = block_matrix([[matrix(ZZ, 1, 1, [3]), matrix(b)], [matrix(ZZ, 64, 1, [0] * 64), basis]])
    res = res.LLL(beta = 25)
    e = res[0][1:]
    try:
        s = A \ (b - e)
        return s
    except:
        return None

f = open("output.txt").read()
data = f.split('\n')

B = Matrix(ZZ, 7, 320, list(map(int, data[0].replace('[', '').replace(']', '').split(', '))))
J = Matrix(ZZ, 64, 25, list(map(int, data[1].replace('[', '').replace(']', '').split(', '))))
R = Matrix(ZZ, 65, 1, list(map(int, data[2].replace('[', '').replace(']', '').split(', '))))
iv = long_to_bytes(int(data[3], 16))[:16]
ct = long_to_bytes(int(data[3], 16))[16:]

res = B.LLL()
idx_list = list(cartesian_product([[2, 3, 4, 5, 6] for _ in range(5)]))
op_list = list(cartesian_product([[-1, 0, 1] for _ in range(5)]))
ans = get_A(res, idx_list, op_list)
ans = [i if i>0 else -i for i in ans]
possible_A  = list(map(Matrix, list(itertools.permutations(ans))))
possible_A  = [i.transpose() for i in possible_A]

T = R[:-1].transpose()
V = R[-1]
k = T.transpose().stack(V).transpose()
kk = k.right_kernel_matrix()
kkk = kk.LLL()
b = kkk[0][:-1]

for A in possible_A:
    try:
        AA = Matrix(Zmod(1000), 64, 25, [int(i).__xor__(int(j)) for i,j in zip(A.list(), J.list())])
        res = get_s(AA, b)
        key = sha256(''.join(list(map(str, res))).encode()).digest()
        cipher = AES.new(key, AES.MODE_CBC, iv)
        pt = cipher.decrypt(ct)
        if pt.startswith(b"X-NUCA{"):
            print(pt)
            break
    except:
        continue

b'X-NUCA{Wh4t_Tru1y_i5_l0giC?_Wh0_d3c1des_re4soN_12e8h8vbd82t4e6q}'


**P.S.**

* In order to increase the probability of success in some cases, we can also expand the scope of the solution space by adopting dimensionality reduction.

* The content of the FLAG is a quote from movie *A Beautiful Mind* "I've always believed in numbers and the equations and logics that lead to reason. But after a lifetime of such pursuits, I ask, 'What truly is logic? Who decides reason?'"

**Reference**

<a id="1" href = "https://cims.nyu.edu/~regev/papers/qcrypto.pdf"> [1] Regev, Oded. "On lattices, learning with errors, random linear codes, and cryptography." Journal of the ACM (JACM) 56.6 (2009): 1-40.</a>

<a id="2" href = "https://cims.nyu.edu/~regev/papers/lwesurvey.pdf"> [2] Regev, Oded. "The learning with errors problem." Invited survey in CCC 7 (2010): 30.</a>

<a id = "3" href = "https://www.esat.kuleuven.be/cosic/publications/thesis-267.pdf"> [3] De Meyer, Lauren. "Security of LWE-based cryptosystems." </a>

<a id = "4" href = "https://eprint.iacr.org/2013/839.pdf">[4] Bai, Shi, and Steven D. Galbraith. "Lattice decoding attacks on binary LWE." Australasian Conference on Information Security and Privacy. Springer, Cham, 2014.</a>